In [ ]:
import pandas as pd
import os
import sys

def leer_archivo(ruta_archivo: str):
    # Verificar que el archivo existe
    if not os.path.exists(ruta_archivo):
        sys.exit(f"❌ El archivo no existe: {ruta_archivo}")

    # Detectar la extensión
    extension = os.path.splitext(ruta_archivo)[1].lower()

    try:
        if extension == ".csv":
            df = pd.read_csv(
                ruta_archivo,
                thousands=",",         # ✅ interpreta comas como miles (2022, 20000)
                na_values=["", " "],   # ✅ vacíos como NaN
                keep_default_na=True
            )
        elif extension in [".xls", ".xlsx"]:
            df = pd.read_excel(ruta_archivo)
        elif extension == ".json":
            df = pd.read_json(ruta_archivo)
        else:
            sys.exit(f"⚠️ Tipo de archivo no soportado: {extension}")

        # Mostrar solo las primeras 10 filas
        print(df.head(10))
        return df

    except Exception as e:
        sys.exit(f"❌ Error al leer el archivo: {e}")

In [ ]:
def combinar_dataframes(df1, df2, cols1=None, cols2=None, how="inner"):
    """
    Combina dos DataFrames en base a columnas comunes o especificadas.
    
    Parámetros:
    -----------
    df1 : DataFrame
        Primer DataFrame
    df2 : DataFrame
        Segundo DataFrame
    cols1 : str o list, opcional
        Columna(s) en df1 para la unión
    cols2 : str o list, opcional
        Columna(s) en df2 para la unión
    how : str, default "inner"
        Tipo de unión: 'left', 'right', 'outer', 'inner'

    Retorna:
    --------
    DataFrame combinado
    """
    if cols1 is None and cols2 is None:
        # Detectar columnas comunes automáticamente
        comunes = list(set(df1.columns) & set(df2.columns))
        if not comunes:
            raise ValueError("❌ No hay columnas en común para combinar.")
        return pd.merge(df1, df2, on=comunes, how=how)

    elif cols1 is not None and cols2 is not None:
        # Unión con columnas diferentes
        return pd.merge(df1, df2, left_on=cols1, right_on=cols2, how=how)

    else:
        raise ValueError("❌ Debes pasar ambas listas de columnas (cols1 y cols2) o ninguna.")

In [ ]:
import matplotlib.pyplot as plt

# Ruta del archivo
ruta = "./data/uso_estaciones.csv"
df = leer_archivo(ruta)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Asegurarse que las columnas de 'Consumo Primera Mitad' y 'Consumo Segunda Mitad' sean numéricas
df["CONSUMO PRIMERA MITAD DEL MES"] = pd.to_numeric(df["CONSUMO PRIMERA MITAD DEL MES"], errors="coerce")
df["CONSUMO SEGUNDA MITAD DEL MES"] = pd.to_numeric(df["CONSUMO SEGUNDA MITAD DEL MES"], errors="coerce")
df["TOTAL DE CONSUMO"] = pd.to_numeric(df["TOTAL DE CONSUMO"], errors="coerce")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

plt.figure(figsize=(18, 6))

# Graficar línea con puntos grandes y color distintivo
plt.plot(df["MES"] + " " + df["AÑO"].astype(str), df["TOTAL DE CONSUMO"], 
        marker='o', linestyle='-', color='royalblue', markersize=6)

plt.title("Consumo Total Mensual", fontsize=16, fontweight='bold')
plt.xlabel("Meses", fontsize=12)
plt.ylabel("Consumo Total", fontsize=12)

# Mejorar etiquetas eje X
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.tick_params(axis='x', which='major', pad=10)

# Formato de número en eje Y con separadores de miles
plt.gca().yaxis.set_major_formatter(mtick.StrMethodFormatter('{x:,.0f}'))

# Mostrar grilla ligera
plt.grid(True, linestyle='--', alpha=0.5)

# Etiquetas en puntos, pero evitar superposición excesiva
for i, txt in enumerate(df["TOTAL DE CONSUMO"]):
    plt.annotate(f'{int(txt):,}', 
                (i, df["TOTAL DE CONSUMO"][i]), 
                textcoords="offset points", xytext=(0, 8), rotation=50, ha='center', fontsize=8, color='darkblue')

plt.tight_layout()
plt.show()

In [ ]:
# 3. Graficar el consumo promedio anual por año con barras llamativas
df["AÑO"] = df["AÑO"].astype(str)
consumo_anual = df.groupby("AÑO")["TOTAL DE CONSUMO"].mean()

# Graficar el consumo promedio anual
plt.figure(figsize=(10, 6))
bars = plt.bar(range(len(consumo_anual)), consumo_anual.values,
                color=['mediumorchid', 'darkorange', 'mediumseagreen'],
                edgecolor='black')

plt.title("Consumo Promedio Anual", fontsize=16, fontweight='bold')
plt.xlabel("Año", fontsize=12)
plt.ylabel("Consumo Promedio", fontsize=12)
plt.xticks(ticks=range(len(consumo_anual)), labels=consumo_anual.index, fontsize=12)
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

# Etiquetas encima de cada barra
for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height + (max(consumo_anual.values) * 0.02),
            f'{height:.0f}', ha='center', fontsize=9, color='#333')

plt.show()
